In [6]:
import pandas as pd
import ast
import re
from glob import glob

# 1) preprocessing_all.csv 불러오기
df_pre = pd.read_csv('preprocessing_all.csv')

# 2) explode_row 함수 (더 견고하게 고친 버전)
def explode_row(row):
    num_cols = [
        '매출액증감',
        '관객수증감(전일대비)',
        '매출액',
        '관객수',
        '스크린수',
        '상영횟수'
    ]
    seqs = {}
    # — 숫자 칼럼들: 리스트 문자열이면 literal_eval, 아니면 단일값을 리스트에 담기
    for col in num_cols:
        cell = row[col]
        if isinstance(cell, str) and cell.strip().startswith('['):
            try:
                lst = ast.literal_eval(cell)
            except Exception:
                lst = [cell]
        else:
            lst = [cell]
        seqs[col] = lst

    # — 조회일: 문자열에 리스트 형태로 들어있으면 regex, 아니면 단일 날짜로
    cell_dates = row['조회일']
    if isinstance(cell_dates, str) and cell_dates.strip().startswith('['):
        dates = re.findall(r"(\d{4}-\d{2}-\d{2})", cell_dates)
    elif isinstance(cell_dates, str):
        # 간혹 "'2020-08-04 00:00:00'" 같은 단일 문자열인 경우
        m = re.search(r"(\d{4}-\d{2}-\d{2})", cell_dates)
        dates = [m.group(1)] if m else []
    elif isinstance(cell_dates, pd.Timestamp):
        dates = [cell_dates.strftime('%Y-%m-%d')]
    else:
        # 이미 파싱된 리스트나 기타
        try:
            dates = list(cell_dates)
        except Exception:
            dates = [str(cell_dates)]

    # 3) 길이를 맞춰서 explode
    length = max(len(dates), max(len(v) for v in seqs.values()))
    records = []
    for i in range(length):
        dt = dates[i] if i < len(dates) else None
        rec = {
            '영화명': row['영화명'],
            '조회일': pd.to_datetime(dt) if dt else pd.NaT
        }
        for col in num_cols:
            rec[col] = seqs[col][i] if i < len(seqs[col]) else None
        records.append(rec)
    return pd.DataFrame(records)

# 4) explode & BASE_YM 생성
df_ts = pd.concat([explode_row(r) for _, r in df_pre.iterrows()],
                  ignore_index=True)
df_ts['BASE_YM'] = df_ts['조회일'].dt.strftime('%Y%m').astype(int)

# 5) 문화지수 데이터 로드
files = glob('문화지수데이터/KC_CLTUR_MOVIE_ACTIVE_IDEX_LIST_*.csv')
df_idx = pd.concat((pd.read_csv(f, dtype={'BASE_YM':int}) for f in files),
                   ignore_index=True)

# 6) 모든 지표를 지역별로 pivot + 컬럼 평탄화
metrics = [c for c in df_idx.columns
           if c not in ['BASE_YM','BASE_YEAR','BASE_MT','CTPRVN_CD','CTPRVN_NM']]
df_wide = df_idx.pivot_table(
    index='BASE_YM',
    columns='CTPRVN_NM',
    values=metrics,
    aggfunc='first'
)
df_wide.columns = [f"{region}_{metric}" for metric, region in df_wide.columns]
df_wide = df_wide.reset_index()

# 7) 최종 merge
df_merged = df_ts.merge(df_wide, on='BASE_YM', how='left')

# 8) 결과 확인
print(df_merged.head())
print(df_merged.columns.tolist())


     영화명        조회일     매출액증감 관객수증감(전일대비)      매출액   관객수 스크린수 상영횟수  BASE_YM  \
0  #살아있다 2020-10-28   2890000         289  2890000   289    2    2   202010   
1  #살아있다 2020-08-30  -1266000        -253  4445000   889    5    5   202008   
2  #살아있다 2020-08-29   3707000         742  5711000  1142    8    8   202008   
3  #살아있다 2020-08-26   2004000         400  2004000   400    3    3   202008   
4  #살아있다 2020-08-11     39000           6    46000     7    1    1   202008   

   강원도_EXPNDTR_PRICE  ...  서울시_STDR_SCRNG_MOVIE_CO  세종시_STDR_SCRNG_MOVIE_CO  \
0       9.091682e+08  ...                234.58333                234.58333   
1       1.625776e+09  ...                234.58333                234.58333   
2       1.625776e+09  ...                234.58333                234.58333   
3       1.625776e+09  ...                234.58333                234.58333   
4       1.625776e+09  ...                234.58333                234.58333   

   울산시_STDR_SCRNG_MOVIE_CO  인천시_STDR_SCRNG_MOVIE_C

In [7]:
df_merged.to_csv('merged_culture_data.csv', index=False, encoding='utf-8-sig')